In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv('C:\\Users\\M\\Machine_L\\data\\heart_failure_clinical_records_dataset.csv')
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [9]:
df.DEATH_EVENT.unique()

array([1, 0], dtype=int64)

In [10]:
#1. 전처리
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [11]:
#2. X, y 나누기
X = df.drop(columns = ['DEATH_EVENT'])
y = df.DEATH_EVENT

In [13]:
#3. train, test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 2024)

In [14]:
#4. 모델 학습
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [15]:
#5. 예측
pred = lr.predict(X_test)

In [16]:
#6. 모델평가
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89        56
           1       0.68      0.68      0.68        19

    accuracy                           0.84        75
   macro avg       0.79      0.79      0.79        75
weighted avg       0.84      0.84      0.84        75



In [17]:
#SVC

In [18]:
#4. 모델학습
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)

SVC()

In [19]:
#5. 예측
pred2 = svc.predict(X_test)

In [20]:
#6. 모델 평가
from sklearn.metrics import classification_report
print(classification_report(y_test, pred2))

              precision    recall  f1-score   support

           0       0.75      1.00      0.85        56
           1       0.00      0.00      0.00        19

    accuracy                           0.75        75
   macro avg       0.37      0.50      0.43        75
weighted avg       0.56      0.75      0.64        75



C:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# hyper parameter optimization

from sklearn.model_selection import GridSearchCV

params = {'C':[0.001, 0.01, 0.1, 1, 10, 100], 'gamma':[0.00001, 0.0001, 0.01, 0.1, 1, 10, 100]} 
grid = GridSearchCV(SVC(), param_grid = params, refit = True, verbose = True)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'gamma': [1e-05, 0.0001, 0.01, 0.1, 1, 10, 100]},
             verbose=True)

In [22]:
bp = grid.best_params_
bs = grid.best_score_

print('grid.best_params_ : ', bp)
print('grid.best_score_ : ', bs)

grid.best_params_ :  {'C': 100, 'gamma': 0.0001}
grid.best_score_ :  0.665050505050505


In [ ]:
# 앙상블

In [23]:
#4. 모델학습
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.ensemble import VotingClassifier

tree = DecisionTreeClassifier()
log = LogisticRegression(solver='lbfgs', max_iter=1000)
knn = KNeighborsClassifier()
svc = SVC(probability=True)

vot_hard = VotingClassifier(estimators = [('log', log), ('tree', tree), ('knn', knn), ('svc', svc)], voting = 'hard')
vot_soft = VotingClassifier(estimators = [('log', log), ('tree', tree), ('knn', knn), ('svc', svc)], voting = 'soft')

models = [tree, log, knn, svc, vot_hard, vot_soft]

In [24]:
from sklearn.metrics import accuracy_score

for m in models:
  m.fit(X_train, y_train)
  pred = m.predict(X_test)
  print(m.__class__.__name__, accuracy_score(y_test, pred))

DecisionTreeClassifier 0.7333333333333333
LogisticRegression 0.84
KNeighborsClassifier 0.6933333333333334
SVC 0.7466666666666667
VotingClassifier 0.8133333333333334
VotingClassifier 0.8


In [25]:
#배깅

In [26]:
from sklearn.ensemble import BaggingClassifier

bag = BaggingClassifier()
bag.fit(X_train, y_train)
pred = bag.predict(X_test)
print(bag.__class__.__name__, accuracy_score(y_test, pred))

BaggingClassifier 0.8


In [27]:
#랜덤포레스트
@@@@@가장 높다@@@@

In [28]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
print(rfc.__class__.__name__, accuracy_score(y_test, pred))

RandomForestClassifier 0.8933333333333333


In [29]:
#Adaboost

In [30]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(X_train, y_train)
pred = ada.predict(X_test)
print(ada.__class__.__name__, accuracy_score(y_test, pred))

AdaBoostClassifier 0.8133333333333334


In [ ]:
# GradientBoosting

In [31]:
from sklearn.ensemble import GradientBoostingClassifier

grad = GradientBoostingClassifier()
grad.fit(X_train, y_train)
pred = grad.predict(X_test)
print(grad.__class__.__name__, accuracy_score(y_test, pred))

GradientBoostingClassifier 0.88


In [32]:
#xgboost

In [33]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)
pred = grad.predict(X_test)
print(xgb.__class__.__name__, accuracy_score(y_test, pred))

XGBClassifier 0.88


In [34]:
# lightgbm

In [35]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
pred = grad.predict(X_test)
print(lgbm.__class__.__name__, accuracy_score(y_test, pred))

[LightGBM] [Info] Number of positive: 77, number of negative: 147
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002078 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 282
[LightGBM] [Info] Number of data points in the train set: 224, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.343750 -> initscore=-0.646627
[LightGBM] [Info] Start training from score -0.646627
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
